# Combine Files

- A*

In [1]:
%ls ..\\..\\data\\results\\

 Volume in drive H is rlrson
 Volume Serial Number is 8000-0458

 Directory of H:\Develop\cs4420-project\data\results

05/05/2017  12:40 PM    <DIR>          .
05/05/2017  11:58 AM    <DIR>          ..
05/05/2017  12:41 PM    <DIR>          canonical
05/05/2017  11:58 AM            37,313 disjointPDBHorizontal.csv
05/05/2017  11:58 AM            33,676 disjointPDBMax.csv
05/05/2017  11:58 AM            36,293 disjointPDBVertical.csv
05/05/2017  11:58 AM            55,639 ida-linearConflict.csv
05/05/2017  11:58 AM            57,374 linearConflict.csv
05/05/2017  11:58 AM            53,042 manhattan.csv
05/05/2017  11:58 AM            52,272 NMaxSwap.csv
05/05/2017  11:58 AM            35,273 nonAdditiveFringe.csv
05/05/2017  11:58 AM            33,743 nonAdditiveMax.csv
               9 File(s)        394,625 bytes
               3 Dir(s)  20,635,701,248 bytes free


In [2]:
# Has many pre configured variables 
%run analysis.py

To plot, execute the following:
from matplotlib import pyplot as mplt
from pandas.tools.plotting import parallel_coordinates
from scipy import stats

import seaborn as sns
sns.set_palette('muted')
sns.set_style('ticks')
sns.set_context('notebook')


In [3]:
dfs = [pd.read_csv(x, names=column_names) for x in files]

In [4]:
for var in ['size', 'heur', 'algo']:
    for df in dfs:
        print (df[var].unique())

['3x3' '4x4']
['3x3' '4x4']
['3x3' '4x4']
['3x3' '4x4']
['3x3' '4x4']
['3x3' '4x4']
['3x3' '4x4']
['3x3' '4x4']
['3x3' '4x4']
[' disjointPDBHorizontal']
[' disjointPDBVertical']
[' disjointPDBMax']
[' linearConflict']
[' linearConflict']
[' manhattan']
[' NMaxSwap']
[' nonAdditiveMax']
[' nonAdditiveFringe']
[' astar']
[' astar']
[' astar']
[' astar']
[' ida']
[' astar']
[' astar']
[' astar']
[' astar']


In [5]:
combined = pd.concat(dfs)
combined.head()

,size,boardID,algo,heur,expanded,branching,cost,time
0,3x3,1.txt,astar,disjointPDBHorizontal,1,3.00,1,4893
1,3x3,2.txt,astar,disjointPDBHorizontal,1,3.00,1,4814
2,3x3,3.txt,astar,disjointPDBHorizontal,2,2.19,2,4723
3,3x3,4.txt,astar,disjointPDBHorizontal,2,1.79,2,4770
4,3x3,5.txt,astar,disjointPDBHorizontal,0,1.00,0,3


In [6]:
# fix the extra space in some strings
combined['heur'] = combined['heur'].str.strip()
combined['algo'] = combined['algo'].str.strip()
combined['size'] = combined['size'].str.strip()
combined['boardID'] = combined['boardID'].str.strip()

In [7]:
# drop rows that will give us trouble later on
# >>	size	boardID	algo	heur	expanded	branching	cost	time
# >>	4840	4x4	23.txt	astar	nonAdditiveFringe		NaN	NaN	NaN

print (combined.size)
combined.dropna(subset=metrics, inplace=True)

combined_size = combined.size
print (combined_size)

48128
48128


In [8]:
# Minus 1 from the branching factor, because we can always go to the root
combined.branching = combined.branching - 1

In [9]:
# reorder the columns 
col_order = ['boardID', 'algo', 'heur', 'size', 'expanded', 'branching', 'time', 'cost']
combined = combined[col_order]

---

Combined results.

In [10]:
# write out the completed file to disk
combined.to_csv(combined_f, index=False)

---

Parallel Coordinates file.

In [11]:
# create a outlier-filtered dataset for the parallel coordinates
parcords_df = combined[np.abs(combined.time-combined.time.mean())<=(3*combined.time.std())]
print ('Number of rows dropped: {}'.format((combined_size - parcords_df.size)))

Number of rows dropped: 680


In [12]:
# write out the par cords to disk
parcords_df.to_csv(parcord_f, index=False)

---

Number of records.

In [13]:
num_records_df = combined[['algo', 'heur', 'size', 'boardID']].groupby(['algo', 'heur', 'size']).count()
num_records_df

boardID
algo  heur                  size         
astar NMaxSwap              3x3       426
                            4x4       440
      disjointPDBHorizontal 3x3       426
                            4x4        84
      disjointPDBMax        3x3       426
                            4x4        83
      disjointPDBVertical   3x3       426
                            4x4        84
      linearConflict        3x3       426
                            4x4       441
      manhattan             3x3       426
                            4x4       441
      nonAdditiveFringe     3x3       426
                            4x4        84
      nonAdditiveMax        3x3       426
                            4x4        84
ida   linearConflict        3x3       426
                            4x4       441

In [14]:
num_records_df.to_csv((canonical_dir + 'number-experiments.csv'))

---

Algorithm Comparison.

In [15]:
algos_df = combined[combined.heur == 'linearConflict']
algos_df.to_csv(algos_f, index=False)